## Генерация данных

In [ ]:
import matplotlib.pyplot as plt

import numpy as np

import mne
from mne.datasets import sample

In [ ]:
# Загружаем данные

data_path = sample.data_path()
subjects_dir = data_path / "subjects"
subject = "sample"

evoked_fname = data_path / "MEG" / subject / "sample_audvis-ave.fif"
info = mne.io.read_info(evoked_fname)
tstep = 1.0 / info["sfreq"]

fwd_fname = data_path / "MEG" / subject / "sample_audvis-meg-eeg-oct-6-fwd.fif"
fwd = mne.read_forward_solution(fwd_fname)
src = fwd["src"]

In [ ]:
# Реализация полезных функций

def create_label(brain_area: str, subject: str, subjects_dir: str):
    # Выбираем область мозга
    selected_label = mne.read_labels_from_annot(
        subject, regexp=brain_area, subjects_dir=subjects_dir
    )[0]

    location = "center"  # центр региона
    extent = 10.0  # 10мм радиус выбора

    # Выбираем соответствующие источники
    label = mne.label.select_sources(
        subject, selected_label, location=location, extent=extent, subjects_dir=subjects_dir
    )
    
    return label

def create_waves(n_waves: int, tstep: int, n_samples: int = 100):
    time = np.arange(n_samples) * tstep  
    waves = np.zeros((n_waves, n_samples)) 
    
    for i in range(n_waves):
        num_terms = np.random.randint(1, 7) 
        frequencies = np.random.uniform(30, 100, num_terms)
        phases = np.random.uniform(0, 1 * np.pi, num_terms) 
        
        wave = np.sum([np.sin(2.0 * np.pi * f * time + p) for f, p in zip(frequencies, phases)], axis=0)
        waves[i] = wave * 10e-9

    return waves

def create_events(n_samples: int, num_events: tuple = (50, 70), start=100, end=10000):
    events_list = []
    for i in range(n_samples):
        n_events = np.random.randint(num_events[0], num_events[1], 1)[0]
        
        events = np.zeros((n_events, 3), int)
        events[:, 0] = np.sort(np.random.randint(start, end - 100, n_events))  
        events[:, 2] = 1
        
        events_list.append(events)
        
    return events_list

def get_sensitive_electrodes(fwd, label):
    # Определяем какие электроды наиболее чувствительны к источникам

    fwd_fixed = mne.convert_forward_solution(fwd, force_fixed=True, surf_ori=True)

    vertices = np.concatenate([pole["vertno"] for pole in src])
    label_indices = np.where(np.isin(vertices, label.vertices))[0]

    picks_eeg = mne.pick_types(fwd["info"], meg=False, eeg=True)

    leadfield_subset = fwd_fixed['sol']['data'][:, label_indices]  # Оставляем только нужные источники
    leadfield_subset_eeg = leadfield_subset[picks_eeg]

    sensor_names = np.array(fwd_fixed['info']['ch_names'])[picks_eeg]

    sensor_sensitivity = np.sum(leadfield_subset_eeg, axis=1)  # Суммарная чувствительность
    sensor_sensitivity_dict = dict(zip(sensor_names, sensor_sensitivity))

    top_sensors = sorted(sensor_sensitivity_dict.items(), key=lambda x: x[1], reverse=True)
    
    return top_sensors[0]

    # # Выводим топ-5 наиболее чувствительных сенсоров
    # for sensor, sensitivity in top_sensors[:5]:
    #     print(f"Сенсор {sensor}: чувствительность {sensitivity:.4f}")


In [ ]:
# Выбранные области мозга
brain_areas = ["caudalmiddlefrontal-lh", "caudalmiddlefrontal-rh", 
               'lateraloccipital-lh', 'lateraloccipital-rh', 
               'postcentral-lh', 'bankssts-lh',
               'supramarginal-lh', 'inferiorparietal-rh',
               'insula-lh', 'insula-rh',
               'precuneus-lh', 'posteriorcingulate-rh',
               'lingual-lh', 'lingual-rh',
               'frontalpole-lh', 'frontalpole-rh']

data_per_area = 125

output_path = "../data/raw_eeg/"

In [ ]:
# Генерация данных
for i, brain_area in enumerate(brain_areas):
    print(f"brain area idx: {i}")
    label = create_label(brain_area=brain_area, subject=subject, subjects_dir=subjects_dir)
    waves = create_waves(data_per_area, tstep=tstep)
    events_list = create_events(data_per_area)
    electrode = get_sensitive_electrodes(fwd=fwd, label = label)
    
    for j in range(data_per_area):
        
        if (j % 5 == 0):
            print(f"{j} starting")
        events = events_list[j]
        source_time_series = waves[j]
        
        # Симуляция работы источника
        source_simulator = mne.simulation.SourceSimulator(src, tstep=tstep)
        source_simulator.add_data(label, source_time_series, events)
        
        # Проекция сигналов на электроды
        raw = mne.simulation.simulate_raw(info, source_simulator, forward=fwd)

        # Добавление шума
        cov = mne.make_ad_hoc_cov(raw.info)
        mne.simulation.add_noise(raw, cov, iir_filter=[0.2, -0.2, 0.04])
        
        # Сохранение данных
        raw.resample(600, npad="auto")
        filename = f"{2000 + i * data_per_area + j + 1:04d}_eeg.fif"
    
        raw.save(output_path + filename, overwrite = True, picks='eeg')
        print(f"saved {filename}")
    

In [ ]:
# Создание словаря валидаций
validation = {}
for k in range(2):
    for i, brain_area in enumerate(brain_areas):
        print(f"brain area idx: {i}")
        label = create_label(brain_area=brain_area, subject=subject, subjects_dir=subjects_dir)
        electrode = get_sensitive_electrodes(fwd=fwd, label = label)
        # print(f"{i}. {brain_area}:\t\t{electrode}")  
        for j in range(data_per_area):
            filename = f"{2000 * k + i * data_per_area + j + 1:04d}_eeg.fif"
            validation[filename] = brain_area  

In [ ]:
#  Сохранение валидацй

import json

with open("validation.json", "w") as f:
    json.dump(validation, f, indent=4) 
    
sorted_values = np.array([value for key, value in sorted(validation.items())])
np.save("validation_areas.npy", sorted_values)